In [1]:
import f3dasm
import numpy as np
import time
from multiprocessing import Pool

In [8]:
dim = 2
iterations = 50
realizations = 10
bounds = np.array([[-1.0, 1.0], [-1.0, 1.0]])
hyperparameters={}

design = f3dasm.make_nd_continuous_design(bounds=bounds, dimensions=dim)

function = f3dasm.functions.Ackley(dimensionality=dim, noise=False, scale_bounds=bounds)

data = f3dasm.Data(designspace=design)
optimizer = f3dasm.optimization.CMAES(data=data, hyperparameters=hyperparameters)
sampler = f3dasm.sampling.LatinHypercubeSampling(doe=design)


# all_data = f3dasm.run_multiple_realizations(optimizer=optimizer, 
# function=function,
# sampler=sampler,
# iterations=iterations,
# realizations=realizations
# )



In [10]:
args = {
'realizations': realizations,
'optimizer': optimizer, 
'function': function,
'sampler': sampler,
'iterations': iterations,


}
f3dasm.run_multiple_realizations(**args)

[Data(designspace=DesignSpace(input_space=[ContinuousParameter(name='x0', lower_bound=-1.0, upper_bound=1.0, type='float'), ContinuousParameter(name='x1', lower_bound=-1.0, upper_bound=1.0, type='float')], output_space=[ContinuousParameter(name='y', lower_bound=-inf, upper_bound=inf, type='float')]), data=             input                   output
                 x0            x1          y
 0     6.930749e-01 -3.084881e-02  19.806444
 1    -5.333847e-01  5.673071e-01  20.035312
 2     2.314243e-01  1.733977e-01  16.904582
 3     1.694220e-01 -1.816147e-01  15.441053
 4     8.184458e-01 -2.735004e-01  21.106759
 ...            ...           ...        ...
 1525  9.711650e-08 -7.721011e-08   0.000011
 1526 -6.244350e-07 -3.877651e-07   0.000067
 1527 -4.969007e-07  2.539621e-07   0.000051
 1528  2.938984e-07 -1.449603e-07   0.000030
 1529  8.308338e-07 -1.182080e-06   0.000131
 
 [1530 rows x 3 columns]),
 Data(designspace=DesignSpace(input_space=[ContinuousParameter(name='x0', lower_

In [11]:
"""Run multiple realizations of the same algorithm on a benchmark function"""
start_t = time.perf_counter()

seed = np.random.randint(low=0, high=1e5)
all_data = []

for _ in range(realizations):
    data = f3dasm.run_optimization(
        optimizer=optimizer, function=function, sampler=sampler, iterations=iterations, seed=seed
    )
    all_data.append(data)

    # Increase seed
    seed += 1

end_t = time.perf_counter()

total_duration = end_t - start_t
print(f"Optimization took {total_duration:.2f}s total")

Optimization took 5.45s total


In [4]:
"""Run multiple realizations of the same algorithm on a benchmark function"""
start_t = time.perf_counter()

args = [(optimizer, function,  sampler, iterations, np.random.randint(low=0, high=1e5)) for _ in range(realizations)]


with Pool() as pool:
    results = pool.starmap(f3dasm.run_optimization, args)
    
end_t = time.perf_counter()

total_duration = end_t - start_t
print(f"Optimization took {total_duration:.2f}s total")

Optimization took 1.73s total


In [5]:
results

[Data(designspace=DesignSpace(input_space=[ContinuousParameter(name='x0', lower_bound=-1.0, upper_bound=1.0, type='float'), ContinuousParameter(name='x1', lower_bound=-1.0, upper_bound=1.0, type='float')], output_space=[ContinuousParameter(name='y', lower_bound=-inf, upper_bound=inf, type='float')]), data=             input                   output
                 x0            x1          y
 0    -3.775276e-01  2.284255e-01  18.714142
 1     3.683647e-01 -1.296051e-01  17.783462
 2     9.864181e-01 -1.820703e-01  21.708502
 3     8.509007e-01 -8.995671e-01  21.448672
 4     6.648060e-01  4.891698e-01  21.542960
 ...            ...           ...        ...
 1525  1.696379e-07  3.930450e-08   0.000016
 1526 -7.608602e-07 -2.171104e-07   0.000072
 1527 -4.476736e-07  5.402385e-07   0.000064
 1528  3.847700e-07 -7.573220e-08   0.000035
 1529  7.557891e-07 -1.423690e-06   0.000146
 
 [1530 rows x 3 columns]),
 Data(designspace=DesignSpace(input_space=[ContinuousParameter(name='x0', lower_

In [6]:
"""Run multiple realizations of the same algorithm on a benchmark function"""
start_t = time.perf_counter()

args = [(optimizer, function,  sampler, iterations, np.random.randint(low=0, high=1e5)) for _ in range(realizations)]


with Pool() as pool:
    results = pool.starmap_async(f3dasm.run_optimization, args)
    
end_t = time.perf_counter()

total_duration = end_t - start_t
print(f"Optimization took {total_duration:.2f}s total")

Optimization took 0.07s total


In [7]:
results

In [ ]:
func.dfdx(np.array([0.5, 0.8]))

In [ ]:
if dim == 2:
    fig, ax = func.plot_data(all_data[0], px=100, domain=bounds)
    ax.scatter(func.get_global_minimum(dim)[0][0],func.get_global_minimum(dim)[0][1], s=40, c='r')
    #ax.scatter(-3.19468541,0.89682456, s=40, c='g')
    func.plot(px=100, domain=bounds)

In [ ]:
y = np.array([[0.2,0.1],[0.3,0.4]])
y + np.abs(func.get_global_minimum(2)[1])

In [ ]:
all_data[0].data['input']

In [ ]:
func.scale_bounds, func.input_domain

In [ ]:
import pandas as pd
mean_y = pd.concat([d.get_output_data() for d in all_data], axis=1).mean(axis=1)
std_y = pd.concat([d.get_output_data() for d in all_data], axis=1).std(axis=1)
mean_y

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.figure(), plt.axes()
ax.plot(mean_y)
ax.fill_between(np.arange(len(mean_y)), mean_y-1.96*std_y, mean_y+1.96*std_y, color='b', alpha=.1)
#ax.set_yscale('log')
fig.show()

In [ ]:
func.get_global_minimum(2)

In [ ]:
func.descale_input(np.array([0.0,0.0]))
x = np.array([[0.0,0.0]])

In [ ]:
func.input_domain

In [ ]:
(func.input_domain[:, 1] - func.input_domain[:, 0]) * x + func.input_domain[:, 0]

In [ ]:
(x - func.input_domain[:, 0]) / (func.input_domain[:, 1] - func.input_domain[:, 0])

In [ ]:
x

In [ ]:
func.input_domain[:, 0]

In [ ]:
x - func.input_domain[:, 0]

In [ ]:
(func.input_domain[:, 1] - func.input_domain[:, 0])

In [ ]:
o = np.array([[0.9,0.3]])
o

In [ ]:
func.scale_input(func.descale_input(o))

In [ ]:
func.descale_input(func.scale_input(o))
o

In [ ]:
scale_bounds = [-3., 1.]

In [ ]:
o = np.random.uniform(low=scale_bounds[0], high=scale_bounds[1], size=(1, func.dimensionality))
x = o
x

In [ ]:
(func.input_domain[:, 1] - func.input_domain[:, 0])

In [ ]:
(x - func.scale_bounds[:, 0]) / (func.scale_bounds[:, 1] - func.scale_bounds[0])

In [ ]:
func.offset

In [ ]:
func.get_global_minimum(2)[0]

In [ ]:
(func.scale_bounds[:, 1] - func.scale_bounds[0])